In [1]:
# bridge related stats

In [2]:
import pandas as pd

In [10]:
df = pd.read_csv(r'../data/bridge/bridge_results.csv')

In [14]:
events = pd.read_csv(r'../data/bridge/bridge_status.csv')

In [158]:
events.iloc[0:10]

,bridge,time,event,date,text,newtime,timeobj
0,Fremont,5:09:25 PM,open,2016-01-31,The Fremont Bridge has reopened to traffic - ...,5:09:25 PM,1900-01-01 17:09:25
1,Fremont,4:59:02 PM,closed,2016-01-31,The Fremont Bridge has closed to traffic - 4...,4:59:02 PM,1900-01-01 16:59:02
2,Montlake,4:23:13 PM,open,2016-01-31,The Montlake Bridge has reopened to traffic ...,4:23:13 PM,1900-01-01 16:23:13
3,Montlake,4:17:50 PM,closed,2016-01-31,The Montlake Bridge has closed to traffic - ...,4:17:50 PM,1900-01-01 16:17:50
4,University,4:15:49 PM,open,2016-01-31,The University Bridge has reopened to traffi...,4:15:49 PM,1900-01-01 16:15:49
5,University,4:09:37 PM,closed,2016-01-31,The University Bridge has closed to traffic ...,4:09:37 PM,1900-01-01 16:09:37
6,Ballard,4:06:20 PM,open,2016-01-31,The Ballard Bridge has reopened to traffic - ...,4:06:20 PM,1900-01-01 16:06:20
7,Ballard,4:04:21 PM,closed,2016-01-31,The Ballard Bridge has closed to traffic - 4...,4:04:21 PM,1900-01-01 16:04:21
8,Fremont,3:55:26 PM,open,2016-01-30,The Fremont Bridge has reopened to traffic - ...,3:55:26 PM,1900-01-01 15:55:26
9,Fremont,3:44:55 PM,closed,2016-01-30,The Fremont Bridge has closed to traffic - 3...,3:44:55 PM,1900-01-01 15:44:55


# average closures 

In [99]:
closures = events[events['event'] == 'closed']
closures_group = closures.groupby(['date','bridge']).count()

In [100]:
closures_group['date'] = closures_group.index.get_level_values(0)
closures_group['bridge'] = closures_group.index.get_level_values(1)
# closures_group[closures_group['bridge'] == 'Ballard']

In [101]:
avg_closure_per_day = {}
for bridge in closures_group.groupby('bridge').count().index:
    avg_closure_per_day[bridge] = closures_group[closures_group['bridge'] == bridge].mean()['event']

In [113]:
df = pd.DataFrame(avg_closure_per_day, index=avg_closure_per_day.keys()).T
df = pd.DataFrame(df['Ballard'])
df.columns = ['avg_closures']
df['bridge'] = df.index
df.to_csv('../data/bridge/avg_closures.csv')

In [114]:
# Get more sophisticated later; add average closures by season, time of day, etc

# closures in current day

In [141]:
import datetime
import math
# load file created by bridge_stat.closures_per_day()
recent_df = pd.read_csv(r'../data/bridge/recent_tweets.csv')

In [142]:
recent_df['local_date'] = pd.to_datetime(recent_df['local_date'])

In [143]:
today = datetime.datetime.today()

In [144]:
tweets_today = recent_df[recent_df['local_date'].apply(lambda row:
                       (row.day == today.day) &
                       (row.month == today.month) &
                       (row.year == today.year))
   ]

In [145]:
# Get a count of tweets by bridge 
tweets_today
closures_today = tweets_today[tweets_today['event'] == 'closed']
closures_by_bridge = closures_today.groupby('bridge').count()
closures_by_bridge['bridge'] = closures_by_bridge.index
closures_by_bridge = closures_by_bridge[['bridge','event']]
closures_by_bridge.columns = ['bridge','closures_so_far']

In [152]:
# Compare to average to get anticipated remained on average
remaining_closures_df = pd.merge(closures_by_bridge, df, left_on='bridge', right_on='bridge')

# Round up for average closures
remaining_closures_df['avg_closures'] = remaining_closures_df['avg_closures'].apply(lambda row: math.ceil(row))
remaining_closures_df['closures_remaining'] = remaining_closures_df['avg_closures'] - remaining_closures_df['closures_so_far']
remaining_closures_df['%_closures_remaining'] = remaining_closures_df['closures_remaining']/remaining_closures_df['avg_closures']

In [155]:
# Leave the negatives in: indicates odds less likely 
remaining_closures_df.to_csv('../data/bridge/closure_estimate.csv')

In [ ]:
#3,7,8,4,2